In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import random as r
import math
import keras
import cv2

from PIL import Image
from keras.preprocessing import image
from keras.utils import np_utils, plot_model

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.python.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
def get_files_withsub(file_dir, train_size):

    images = []   
    subfolders = []
    train_array = []
    test_array = []
    train_img_list = []
    test_img_list = []
    train_all_img = []
    test_all_img = []
 
    for dirPath, dirNames, fileNames in os.walk(file_dir):
        
        names = []
        for name in fileNames:
            names.append(os.path.join(dirPath, name))
        for name in dirNames:
            subfolders.append(os.path.join(dirPath, name))
        r.shuffle(names)
        if names != []:
            images.append(names)

    mincount = float("Inf")
    for num_folder in subfolders:
        n_img = len(os.listdir(num_folder))
        
        if n_img < mincount:
            mincount = n_img

    for i in range(len(images)):
        images[i] = images[i][0:mincount]
        train_img_list.append([])
        test_img_list.append([])
        train_num = math.ceil(len(images[i]) * train_size)
        train_img_list[i] = images[i][0:train_num]
        test_img_list[i] = images[i][train_num:]

    for i in range(len(images)):
        train_all_img.extend(train_img_list[i])

    for i in range(len(images)):
        test_all_img.extend(test_img_list[i])

    train_labels = []
    test_labels = []
    for count in range(len(subfolders)):
        train_labels = np.append(train_labels, len(train_img_list[0]) * [count])
    
    for count in range(len(subfolders)):
        test_labels = np.append(test_labels, len(test_img_list[0]) * [count])
            
    train_array = np.array([train_all_img, train_labels])
    train_array = train_array[:, np.random.permutation(train_array.shape[1])].T
    test_array = np.array([test_all_img, test_labels])
    test_array = test_array[:, np.random.permutation(test_array.shape[1])].T
    
    train_img = list(train_array[:, 0])
    train_labels = list(train_array[:, 1])
    train_labels = [int(float(i)) for i in train_labels]
    
    test_img = list(test_array[:, 0])
    test_labels = list(test_array[:, 1])
    test_labels = [int(float(i)) for i in test_labels]
    
    # 返回图片路径列表和对应标签列表
    return train_img, train_labels, test_img, test_labels

In [3]:
file_dir = 'PIC_database_downsized'
IMAGE_SIZE    = (399, 299)
NUM_CLASSES   = 5
BATCH_SIZE    = 8
FREEZE_LAYERS = 2
NUM_EPOCHS    = 10000

In [4]:
train_img, train_labels, test_img, test_labels = get_files_withsub(file_dir, train_size=0.9)

print(len(train_img), len(test_img))
print ("Done!")

5865 650
Done!


In [5]:
train_images = []
for img in train_img:
    img = Image.open(img)
    img = image.img_to_array(img)
    train_images.append(img)
    
x_train = np.array(train_images)
y_train = np.array(train_labels)

In [6]:
test_images = []
for img in test_img:
    img = Image.open(img)
    img = image.img_to_array(img)
    test_images.append(img)

x_test = np.array(test_images)
y_test = np.array(test_labels)

In [7]:
x_train /= 255
x_test /= 255
y_train_onehot = np_utils.to_categorical(y_train)
y_test_onehot = np_utils.to_categorical(y_test)

In [8]:
model = InceptionResNetV2(include_top=False,
                          weights='imagenet',
                          input_tensor=None,
                          input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
x = model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
model_final = Model(inputs = model.input, outputs = output_layer)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
for layer in model_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

model_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
print(model_final.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 399, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 199, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 199, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 199, 149, 32) 0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
conv2d_1 (

In [12]:
train_history = model_final.fit(x = x_train, 
                                y = y_train_onehot, validation_split = 0.1, 
                                epochs = NUM_EPOCHS, batch_size = BATCH_SIZE,
                                verbose = 1)

Train on 5278 samples, validate on 587 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10000
   8/5278 [..............................] - ETA: 8:37:52 - loss: 1.1648 - acc: 0.6250

KeyboardInterrupt: 

In [ ]:
scores = model_final.evaluate(x_test,y_test_onehot)
print()
print('accuracy=',scores[1])

In [ ]:
def show_train_history(train_history, title, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title(title)
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc = 'upper left')
    plt.show()

show_train_history(train_history, 'Accuracy', 'acc', 'val_acc')
show_train_history(train_history, 'Loss', 'loss', 'val_loss')

In [ ]:
import pandas as pd
prediction = model.predict_classes(x_test)
pd.crosstab(y_test, prediction, rownames=['label'], colnames=['predict'])

In [ ]:
model.save('models/m1/pre/model-inception_resnet_v2-final.h5')

In [ ]:
model.save_weights('models/m1/pre/model-inception_resnet_v2-final_weight.h5')